In [40]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
from bs4 import BeautifulSoup
import urllib.request

## Import all columns from the webpage

In [41]:
def extract_team_data(team_name = None, year_start = None, year_end = None):
    teams = team_name
    print(teams)
    years = np.arange(year_start,year_end+1)
    sub_df = pd.DataFrame()

    for team in teams:
        print("Scraping data for: ", team, "from year:", year_start, "to year:", year_end)
        for year in years:
            try:
                year = str(year)
                try:
                    url = 'https://www.baseball-reference.com/teams/' + team + '/' + year + '-schedule-scores.shtml'
                    page = urllib.request.urlopen(url)
                except Exception:
                    pass
                soup = BeautifulSoup(page, 'html.parser')
                table = soup.find("table", {'id': 'team_schedule'})
                table_rows = table.find_all('tr')
                output = []
                for tr in table_rows:
                    td = tr.find_all('td')
                    row = [tr.text for tr in td]
                    output.append(row)
                df = pd.DataFrame(output)
                #print(df.head())
                df = df[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]
                df.columns = ['date', 'team', 'home-away-indicator', 'opponent',  'win-loss-tie', 'runs',
                              'runs_allowed', 'innings', 'record', 'rank', 'gb', 'win', 'loss', 'save',
                              'time', 'day_night', 'attendance', 'cLI', 'streak']
                df.dropna(how='all', axis=0, inplace=True)
                df['year'] = int(year)
                df['home_team'] = team
                df.drop(columns= ['team'], inplace=True)
                sub_df = pd.concat([sub_df, df], axis=0)
            except Exception as e:
                print(year)
                print(team)
                print(e)
                pass
    return sub_df

#Adapted From:
#https://github.com/OlivierLej/DataIsTheNewOil/blob/4b8bc4d5e56a7c5442843cf2ad1d0a0971f0945e/scraping_baseballreference.py


In [42]:
def partition_home_games(data = None):
    home_games = sub_df.loc[sub_df['xhome-away-indicator' ]== '']
    return home_games

def partition_away_games(data=None):
    away_games = sub_df.loc[sub_df['home-away-indicator' ]== '@']
    return away_games

## Fundamental data cleanup on columns

In [43]:
data = extract_team_data(team_name=['NYM', 'ATL', 'PHI',
                                     'WSN', 'MIA', 'MIL',
                                     'CIN', 'CHC', 'STL',
                                     'PIT', 'SFG', 'LAD',
                                     'SDP', 'COL', 'ARI',
                                     'BOS', 'TBR', 'NYY',
                                     'TOR', 'BAL', 'CHW',
                                     'CLE', 'DET', 'MIN',
                                     'KCR', 'HOU', 'OAK',
                                     'SEA', 'LAA', 'TEX'], 
                                     year_start=2008,year_end=2019)

['NYM', 'ATL', 'PHI', 'WSN', 'MIA', 'MIL', 'CIN', 'CHC', 'STL', 'PIT', 'SFG', 'LAD', 'SDP', 'COL', 'ARI', 'BOS', 'TBR', 'NYY', 'TOR', 'BAL', 'CHW', 'CLE', 'DET', 'MIN', 'KCR', 'HOU', 'OAK', 'SEA', 'LAA', 'TEX']
Scraping data for:  NYM from year: 2008 to year: 2019
Scraping data for:  ATL from year: 2008 to year: 2019
Scraping data for:  PHI from year: 2008 to year: 2019
Scraping data for:  WSN from year: 2008 to year: 2019
Scraping data for:  MIA from year: 2008 to year: 2019
2008
MIA
'NoneType' object has no attribute 'find_all'
2009
MIA
'NoneType' object has no attribute 'find_all'
2010
MIA
'NoneType' object has no attribute 'find_all'
2011
MIA
'NoneType' object has no attribute 'find_all'
Scraping data for:  MIL from year: 2008 to year: 2019
Scraping data for:  CIN from year: 2008 to year: 2019
Scraping data for:  CHC from year: 2008 to year: 2019
Scraping data for:  STL from year: 2008 to year: 2019
Scraping data for:  PIT from year: 2008 to year: 2019
Scraping data for:  SFG from 

In [44]:
data.head()

,date,home-away-indicator,opponent,win-loss-tie,runs,runs_allowed,innings,record,rank,gb,win,loss,save,time,day_night,attendance,cLI,streak,year,home_team
1,"Monday, Mar 31",@,FLA,W,7,2,,1-0,2,0.5,Santana,Hendrickson,,2:53,D,"38,308",1.05,+,2008,NYM
3,"Tuesday, Apr 1",@,FLA,L-wo,4,5,10,1-1,2,1.0,Miller,Wise,,3:31,N,"15,117",1.16,-,2008,NYM
4,"Wednesday, Apr 2",@,FLA,W,13,0,,2-1,2,1.0,Perez,Miller,,2:55,N,"13,720",1.10,+,2008,NYM
5,"Saturday, Apr 5",@,ATL,L,5,11,,2-2,2,0.5,Hudson,Maine,,3:27,D,"36,130",1.11,-,2008,NYM
6,"Sunday, Apr 6",@,ATL,L,1,3,,2-3,3,0.5,Smoltz,Santana,Soriano,2:35,D,"39,414",1.11,--,2008,NYM


In [45]:
data.attendance = data.attendance.str.replace(',','')
data.attendance = pd.to_numeric(data['attendance'])

### Split out the date information into separate columns

In [46]:
date_thangs = data.date.str.split(" ", expand=True)
data['day'] = date_thangs[0].str.replace(",", " ").str.strip()
data.day.unique()
data['month'] = date_thangs[1]
data['num-date'] = date_thangs[2]
data['multi-game'] = date_thangs[3]
data.drop(columns=['date'], inplace=True)

In [47]:
#Convert the wins and losses into separate columns

win_loss = data['record'].str.split("-")
win_loss = pd.DataFrame(win_loss)
win_loss

wins = []
losses = []
for record in data.record:
    win, loss = record.split('-')
    wins.append(int(win))
    losses.append(int(loss))

data['wins'] = wins
data['losses'] = losses
data.info()

data.head(30)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57663 entries, 1 to 168
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   home-away-indicator  57663 non-null  object 
 1   opponent             57663 non-null  object 
 2   win-loss-tie         57663 non-null  object 
 3   runs                 57663 non-null  object 
 4   runs_allowed         57663 non-null  object 
 5   innings              57663 non-null  object 
 6   record               57663 non-null  object 
 7   rank                 57663 non-null  object 
 8   gb                   57663 non-null  object 
 9   win                  57663 non-null  object 
 10  loss                 57663 non-null  object 
 11  save                 57663 non-null  object 
 12  time                 57663 non-null  object 
 13  day_night            57663 non-null  object 
 14  attendance           57444 non-null  float64
 15  cLI                  57663 non-null  o

,home-away-indicator,opponent,win-loss-tie,runs,runs_allowed,innings,record,rank,gb,win,...,cLI,streak,year,home_team,day,month,num-date,multi-game,wins,losses
1,@,FLA,W,7,2,,1-0,2,0.5,Santana,...,1.05,+,2008,NYM,Monday,Mar,31,None,1,0
3,@,FLA,L-wo,4,5,10,1-1,2,1.0,Miller,...,1.16,-,2008,NYM,Tuesday,Apr,1,None,1,1
4,@,FLA,W,13,0,,2-1,2,1.0,Perez,...,1.10,+,2008,NYM,Wednesday,Apr,2,None,2,1
5,@,ATL,L,5,11,,2-2,2,0.5,Hudson,...,1.11,-,2008,NYM,Saturday,Apr,5,None,2,2
6,@,ATL,L,1,3,,2-3,3,0.5,Smoltz,...,1.11,--,2008,NYM,Sunday,Apr,6,None,2,3
7,,PHI,L,2,5,,2-4,3,1.5,Moyer,...,1.07,---,2008,NYM,Tuesday,Apr,8,None,2,4
8,,PHI,W,8,2,,3-4,2,1.5,Pelfrey,...,1.04,+,2008,NYM,Wednesday,Apr,9,None,3,4
9,,PHI,W-wo,4,3,12,4-4,2,1.5,Sosa,...,1.09,++,2008,NYM,Thursday,Apr,10,None,4,4
10,,MIL,W,4,2,,5-4,2,1.5,Figueroa,...,1.03,+++,2008,NYM,Friday,Apr,11,None,5,4
11,,MIL,L,3,5,,5-5,2,1.5,Sheets,...,1.03,-,2008,NYM,Saturday,Apr,12,None,5,5


In [48]:
# convert streak column to integer value
data.streak.fillna('0', inplace=True)
data.streak = [len(x) if '+' in x else -len(x) if '-' in x else 0 for x in data.streak]

In [49]:
data.head()

,home-away-indicator,opponent,win-loss-tie,runs,runs_allowed,innings,record,rank,gb,win,...,cLI,streak,year,home_team,day,month,num-date,multi-game,wins,losses
1,@,FLA,W,7,2,,1-0,2,0.5,Santana,...,1.05,1,2008,NYM,Monday,Mar,31,None,1,0
3,@,FLA,L-wo,4,5,10,1-1,2,1.0,Miller,...,1.16,-1,2008,NYM,Tuesday,Apr,1,None,1,1
4,@,FLA,W,13,0,,2-1,2,1.0,Perez,...,1.10,1,2008,NYM,Wednesday,Apr,2,None,2,1
5,@,ATL,L,5,11,,2-2,2,0.5,Hudson,...,1.11,-1,2008,NYM,Saturday,Apr,5,None,2,2
6,@,ATL,L,1,3,,2-3,3,0.5,Smoltz,...,1.11,-2,2008,NYM,Sunday,Apr,6,None,2,3


In [53]:
data.day_night.unique()

array(['D', 'N'], dtype=object)

### Enrich the base dataset by including information about the stadium capacities and the stadium type

In [59]:
stadium_stats = pd.read_csv('team_stadium_data.csv', encoding = "ISO-8859-1", engine='python')
stadium_stats.head()

,Team Acronym,Name,Capacity,Location,Surface,Team,Opened,Distance to center field,Type,Roof type
0,NYY,Yankee Stadium,"46,537[28]","Bronx, New York",Grass,New York Yankees,2009,408 feet (124 m),Retro-classic,Open
1,CHC,Wrigley Field,"41,649[27]","Chicago, Illinois",Grass,Chicago Cubs,1914[nb 6],400 feet (122 m),Jewel box,Open
2,ATL,Truist Park,"41,084[26]","Cumberland, Georgia",Grass,Atlanta Braves,2017,400 feet (122 m),Retro-modern,Open
3,TBR,Tropicana Field,"25,000[25]","St. Petersburg, Florida",Artificial turf,Tampa Bay Rays,1990[nb 5],404 feet (123 m),Modern,Fixed
4,MIN,Target Field,"38,544[24]","Minneapolis, Minnesota",Grass,Minnesota Twins,2010,404 feet (123 m),Retro-modern,Open


### remove odd characters in the capacity column as well as the commas

In [97]:
a = stadium_stats.Capacity.str.split("[")
cap = []
for stadium in a:
    seats = stadium[0]
    #print(seats)
    cap.append(seats)
print(cap)

stadium_stats.Capacity = cap
stadium_stats.Capacity.str.replace(",", "").str.strip()

['46,537', '41,649', '41,084', '25,000', '38,544', '47,929', '49286', '46,847', '34,830', '38,747', '40,209', '45,971', '41,265', '41,339', '41,168', '36742', '37,903', '40615', '42319', '40,300', '37,755', '56,000', '50,144', '41,083', '42,792', '41,922', '48,405', '45,494', '45,517', '41,900']


0     46537
1     41649
2     41084
3     25000
4     38544
5     47929
6     49286
7     46847
8     34830
9     38747
10    40209
11    45971
12    41265
13    41339
14    41168
15    36742
16    37903
17    40615
18    42319
19    40300
20    37755
21    56000
22    50144
23    41083
24    42792
25    41922
26    48405
27    45494
28    45517
29    41900
Name: Capacity, dtype: object

In [93]:
stadium_stats.head()

,Team Acronym,Name,Capacity,Location,Surface,Team,Opened,Distance to center field,Type,Roof type
0,NYY,Yankee Stadium,"46,537","Bronx, New York",Grass,New York Yankees,2009,408 feet (124 m),Retro-classic,Open
1,CHC,Wrigley Field,"41,649","Chicago, Illinois",Grass,Chicago Cubs,1914[nb 6],400 feet (122 m),Jewel box,Open
2,ATL,Truist Park,"41,084","Cumberland, Georgia",Grass,Atlanta Braves,2017,400 feet (122 m),Retro-modern,Open
3,TBR,Tropicana Field,"25,000","St. Petersburg, Florida",Artificial turf,Tampa Bay Rays,1990[nb 5],404 feet (123 m),Modern,Fixed
4,MIN,Target Field,"38,544","Minneapolis, Minnesota",Grass,Minnesota Twins,2010,404 feet (123 m),Retro-modern,Open


In [80]:
b = a[:,0][0]
b

ValueError: Can only tuple-index with a MultiIndex

In [50]:
data.to_pickle(("./mlb_data.pkl"))